In [3]:
from pylab import *
import matplotlib.pyplot as plt
import os
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pandas as pd
import numpy as np
import pyAgrum as gum

#whether to shoot or not 
# Create a Bayesian Network
import pyAgrum as gum

# Create an Influence Diagram
idm = gum.InfluenceDiagram()

# Add chance nodes (random variables)
assist_type = idm.add(gum.LabelizedVariable("assist_type", "assist_type", 3)) # Pass,Cross,Through ball
situation = idm.add(gum.LabelizedVariable("situation", "situation", 3)) # Open Play, Counter Attack, Set Piece
body_part = idm.add(gum.LabelizedVariable("body_part", "body_part", 2)) #Feet ,  Head
location = idm.add(gum.LabelizedVariable("location", "location", 4)) #Central zone:(centre of the box and penalty spot),Wide zone(left-wing, right-wing,left side of the six-yard box, right side of the box, and right side of the six-yard box),Difficult angle zone(difficult angle and long-range, difficult angle on the left, and difficult angle on the right),Long-range zone(long-range, more than 35 yards, and more than 40 yards)

shot_on_target = idm.add(gum.LabelizedVariable("shot_on_target", "shot_on_target", 2)) 
is_goal = idm.add(gum.LabelizedVariable("is_goal", "is_goal", 2)) 
goal_keeper_position = idm.add(gum.LabelizedVariable("goal_keeper_position", "goal_keeper_position", 3)) # Good positioning, avergage poisitioning, Bad positioning(Far Off)
offsideflag = idm.add(gum.LabelizedVariable("offsideflag", "offsideflag", 2)) #FlagUp (No Goal) ,  Flag Down(Goal)




# Add decision node
shooting_decision = idm.addDecisionNode(gum.LabelizedVariable("shooting_decision", "shooting_decision", 2)) # Shoot, Don't Shoot

# Add utility node
# Add utility node
utility = idm.addUtilityNode(gum.LabelizedVariable("utility", "utility", 1)) # Single label for utility value

#
# Add arcs (directed edges) between nodes
idm.addArc(situation, location)
idm.addArc(assist_type, body_part)
idm.addArc(body_part, location)
idm.addArc(location, shot_on_target)
idm.addArc(body_part, shot_on_target)
idm.addArc(assist_type, shot_on_target)
idm.addArc(situation, shot_on_target)

idm.addArc(shot_on_target,is_goal)
idm.addArc(goal_keeper_position,is_goal)
idm.addArc(is_goal, utility)
idm.addArc(shooting_decision,utility )
idm.addArc(offsideflag, is_goal)
# Define conditional probability tables (CPTs)
idm.cpt(offsideflag).fillWith([0.05,0.95]) # flag up, flag down
idm.cpt(goal_keeper_position).fillWith([0.6, 0.3, 0.2])  # good positioning, Average positioning , Bad(Far off )positioning
idm.cpt(assist_type).fillWith([0.5, 0.2, 0.3]) # Pass, Cross, Throughl Ball


idm.cpt(situation).fillWith([0.8, 0.1, 0.1]) # Open Play, Counter Attack, Set Piece

idm.cpt(body_part)[{'assist_type': 0}] = [0.8, 0.2]
idm.cpt(body_part)[{'assist_type': 1}] = [0.3, 0.7]
idm.cpt(body_part)[{'assist_type': 2}] = [0.95, 0.05]
#Situation: Open Play, Body_part: Feet
idm.cpt(location)[{'situation': 0, 'body_part': 0}] = [0.4, 0.4, 0.15, 0.05] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Open Play, Body_part: Head
idm.cpt(location)[{'situation': 0, 'body_part': 1}] = [0.7, 0.25, 0.04, 0.01] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Counter Attack, Body_part: Feet
idm.cpt(location)[{'situation': 1, 'body_part': 0}] = [0.3, 0.5, 0.15, 0.05] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Counter Attack, Body_part: Head
idm.cpt(location)[{'situation': 1, 'body_part': 1}] = [0.6, 0.3, 0.08, 0.02] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Set Piece, Body_part: Feet
idm.cpt(location)[{'situation': 2, 'body_part': 0}] = [0.35, 0.35, 0.2, 0.1] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Set Piece, Body_part: Head
idm.cpt(location)[{'situation': 2, 'body_part': 1}] = [0.8, 0.15, 0.04, 0.01] # Central zone, Wide zone, Difficult angle zone, Long-range zone

# Situation: Open Play, Assist_type: Pass
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 0, 'location': 0}] = [0.1, 0.9]  # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 0, 'location': 1}] = [0.2, 0.8]  # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 0, 'location': 2}] = [0.7, 0.3]  # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 0, 'location': 3}] = [0.95, 0.05]  # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 1, 'location': 0}] = [0.15, 0.85]  # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 1, 'location': 1}] = [0.3, 0.7]  # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 1, 'location': 2}] = [0.9, 0.1]  # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 1, 'location': 3}] = [0.99, 0.01]  # Head, Long-range zone
#       shot_on_target                 Situation: Open Play, Assist_type: Cross
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 0, 'location': 0}] = [0.6, 0.4]  # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 0, 'location': 1}] = [0.4, 0.6]  # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 0, 'location': 2}] = [0.8, 0.2]  # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 0, 'location': 3}] = [0.98, 0.02]  # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 1, 'location': 0}] = [0.05, 0.95]  # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 1, 'location': 1}] = [0.15, 0.85]  # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 1, 'location': 2}] = [0.95, 0.05]  # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 1, 'location': 3}] = [0.99, 0.01]  # Head, Long-range zone
#       shot_on_target               Situation: Open Play, Assist_type: Through ball
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 0, 'location': 0}] = [0.1, 0.9]  # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 0, 'location': 1}] = [0.3, 0.7]  # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 0, 'location': 2}] = [0.85, 0.15]  # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 0, 'location': 3}] = [0.97, 0.03]  # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 1, 'location': 0}] = [0.2, 0.8]  # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 1, 'location': 1}] = [0.4, 0.6]  # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 1, 'location': 2}] = [0.95, 0.05]  # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 1, 'location': 3}] = [0.99, 0.01]  # Head, Long-range zone
#       shot_on_target            Situation: Counter Attack, Assist_type: Pass
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 0, 'location': 0}] = [0.08, 0.92]  # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 0, 'location': 1}] = [0.25, 0.75]  # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 0, 'location': 2}] = [0.8, 0.2]  # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 0, 'location': 3}] = [0.95, 0.05]  # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 1, 'location': 0}] = [0.2, 0.8]  # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 1, 'location': 1}] = [0.4, 0.6]  # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 1, 'location': 2}] = [0.9, 0.1]  # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 1, 'location': 3}] = [0.98, 0.02]  # Head, Long-range zone
#       shot_on_target               Situation: Counter Attack, Assist_type: Cross
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 0, 'location': 0}] = [0.15, 0.85] # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 0, 'location': 1}] = [0.35, 0.65] # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 0, 'location': 2}] = [0.75, 0.25] # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 0, 'location': 3}] = [0.96, 0.04] # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 1, 'location': 0}] = [0.05, 0.95] # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 1, 'location': 1}] = [0.12, 0.88] # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 1, 'location': 2}] = [0.94, 0.06] # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 1, 'location': 3}] = [0.99, 0.01] # Head, Long-range zone
#       shot_on_target                 Situation: Counter Attack, Assist_type: Through ball
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 0, 'location': 0}] = [0.05, 0.95] # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 0, 'location': 1}] = [0.2, 0.8] # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 0, 'location': 2}] = [0.8, 0.2] # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 0, 'location': 3}] = [0.97, 0.03] # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 1, 'location': 0}] = [0.3, 0.7] # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 1, 'location': 1}] = [0.5, 0.5] # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 1, 'location': 2}] = [0.95, 0.05] # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 1, 'location': 3}] = [0.99, 0.01] # Head, Long-range zone
#       shot_on_target             Situation: Set Piece, Assist_type: Pass
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 0, 'location': 0}] = [0.15, 0.85] # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 0, 'location': 1}] = [0.3, 0.7] # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 0, 'location': 2}] = [0.9, 0.1] # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 0, 'location': 3}] = [0.98, 0.02] # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 1, 'location': 0}] = [0.1, 0.9] # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 1, 'location': 1}] = [0.25, 0.75] # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 1, 'location': 2}] = [0.95, 0.05] # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 1, 'location': 3}] = [0.99, 0.01] # Head, Long-range zon
#       shot_on_target              Situation: Set Piece, Assist_type: Cross
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 0, 'location': 0}] = [0.7, 0.3]  # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 0, 'location': 1}] = [0.5, 0.5]  # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 0, 'location': 2}] = [0.85, 0.15]  # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 0, 'location': 3}] = [0.98, 0.02]  # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 1, 'location': 0}] = [0.2, 0.8]  # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 1, 'location': 1}] = [0.6, 0.4]  # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 1, 'location': 2}] = [0.9, 0.1]  # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 1, 'location': 3}] = [0.99, 0.01]  # Head, Long-range zone
#       shot_on_target                 Situation: Set Piece, Assist_type: Through ball
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 0}] = [0.15, 0.85]  # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 1}] = [0.4, 0.6]  # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 2}] = [0.8, 0.2]  # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 3}] = [0.95, 0.05]  # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 0}] = [0.2, 0.8]  # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 1}] = [0.3, 0.7]  # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 2}] = [0.9, 0.1]  # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 3}] = [0.98, 0.02]  # Head, Long-range zone

# Situation: Set Piece, Assist_type: Through ball
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 0}] = [0.15, 0.85]  # Feet, Central zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 1}] = [0.4, 0.6]  # Feet, Wide zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 2}] = [0.8, 0.2]  # Feet, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 3}] = [0.95, 0.05]  # Feet, Long-range zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 0}] = [0.2, 0.8]  # Head, Central zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 1}] = [0.3, 0.7]  # Head, Wide zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 2}] = [0.9, 0.1]  # Head, Difficult angle zone
idm.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 3}] = [0.98, 0.02]  # Head, Long-range zone

idm.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 0, 'offsideflag': 0}] = [1, 0] # No shot on target, good goalkeeper positioning, offside flag up
idm.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 0, 'offsideflag': 1}] = [1, 0] # No shot on target, good goalkeeper positioning, offside flag down
idm.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 1, 'offsideflag': 0}] = [1, 0] # No shot on target, average goalkeeper positioning, offside flag up
idm.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 1, 'offsideflag': 1}] = [1, 0] # No shot on target, average goalkeeper positioning, offside flag down
idm.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 2, 'offsideflag': 0}] = [1, 0] # No shot on target, bad goalkeeper positioning, offside flag up
idm.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 2, 'offsideflag': 1}] = [1, 0] # No shot on target, bad goalkeeper positioning, offside flag down

idm.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 0, 'offsideflag': 0}] = [1, 0] # Shot on target, good goalkeeper positioning, offside flag up
idm.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 0, 'offsideflag': 1}] = [0.6, 0.4] # Shot on target, good goalkeeper positioning, offside flag down
idm.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 1, 'offsideflag': 0}] = [1, 0] # Shot on target, average goalkeeper positioning, offside flag up
idm.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 1, 'offsideflag': 1}] = [0.3, 0.7] # Shot on target, average goalkeeper positioning, offside flag down
idm.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 2, 'offsideflag': 0}] = [1, 0] # Shot on target, bad goalkeeper positioning, offside flag up
idm.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 2, 'offsideflag': 1}] = [0.15, 0.85] # Shot on target, bad goalkeeper positioning, offside flag down

# Define utility table
# Fill in utility values
idm.utility(utility)[{'shooting_decision': 0, 'is_goal': 0}] = 0
idm.utility(utility)[{'shooting_decision': 0, 'is_goal': 1}] = 100
idm.utility(utility)[{'shooting_decision': 1, 'is_goal': 0}] = -50
idm.utility(utility)[{'shooting_decision': 1, 'is_goal': 1}] = 0

idm

goal_list = [
    {"goal": "Zidane's volley", "evidence": {"assist_type": 0, "situation": 0, "body_part": 0, "location": 0, "goal_keeper_position": 0, "offsideflag": 1}},
    {"goal": "Messi's solo run", "evidence": {"assist_type": 0, "situation": 1, "body_part": 0, "location": 2, "goal_keeper_position": 2, "offsideflag": 1}},
    {"goal": "Ronaldo's long-range", "evidence": {"assist_type": 0, "situation": 2, "body_part": 0, "location": 3, "goal_keeper_position": 1, "offsideflag": 1}},
   {"goal": "Salah's goal against Chelsea", "evidence": {"assist_type": 0, "situation": 0, "body_part": 0, "location": 3, "goal_keeper_position": 2, "offsideflag": 1}},
    {"goal": "Maradona's Hand of God", "evidence": {"assist_type": 0, "situation": 1, "body_part": 1, "location": 1, "goal_keeper_position": 0, "offsideflag": 1}},
   {"goal": "Messi's second goal in World Cup final", "evidence": {"assist_type": 0, "situation": 0, "body_part": 0, "location": 0, "goal_keeper_position": 2, "offsideflag": 1} },
 {"goal": "Messi's pass instead of shot", "evidence": {"assist_type": 0, "situation": 0, "body_part": 0, "location": 0, "goal_keeper_position": 1, "offsideflag": 1} },
     {"goal": "Naby keita Long range attempt UCL final 2022", "evidence": {"assist_type": 0, "situation": 1, "body_part": 0, "location": 3, "goal_keeper_position": 1, "offsideflag": 1} }
]
inference_engine = gum.ShaferShenoyLIMIDInference(idm)
shooting_decision_shoot = 0

for goal in goal_list:
    inference_engine.setEvidence({"assist_type": goal['evidence']['assist_type'], "situation": goal['evidence']['situation'], "body_part": goal['evidence']['body_part'], "location":goal['evidence']['location'] , "goal_keeper_position": goal['evidence']['goal_keeper_position'], "offsideflag": goal['evidence']['offsideflag']})
    inference_engine.makeInference()
    expected_utility_potential = inference_engine.posteriorUtility(shooting_decision_shoot)
    expected_utility = expected_utility_potential
    print(f"Expected utility for {goal['goal']} when deciding to shoot:" ,expected_utility[0])





Expected utility for Zidane's volley when deciding to shoot: 36.0
Expected utility for Messi's solo run when deciding to shoot: 17.0
Expected utility for Ronaldo's long-range when deciding to shoot: 1.4000000000000001
Expected utility for Salah's goal against Chelsea when deciding to shoot: 4.250000000000001
Expected utility for Maradona's Hand of God when deciding to shoot: 23.999999999999996
Expected utility for Messi's second goal in World Cup final when deciding to shoot: 76.5
Expected utility for Messi's pass instead of shot when deciding to shoot: 63.0
Expected utility for Naby keita Long range attempt UCL final 2022 when deciding to shoot: 3.5
